<font color='red'>Voor het maken van dit notebook is <i>Keras</i> gebruikt. Hierbij hebben we expliciet de Keras module gebruikt, niet de tensorflow-keras modules. Om deze te installeren kun je  `pip install keras` uitvoeren.</font>

In [7]:
# Import Modules
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Autoencoders
<img src='https://miro.medium.com/max/1574/1*44eDEuZBEsmG_TCAKRI3Kw@2x.png'>

## Wat is een autoencoder?
Een autoencoder is een vorm van Dimensionality Reduction. Hiervan hebben we een eerder voorbeeld gezien (PCA). Een autoencoder is een vorm van een neuraal netwerk, waarbij getraind wordt op de input van het netwerk. Het doel van een autoencoder is om de output zo gelijk mogelijk te houden aan de input, waarbij er een $n$ aantal neuronen in de "bottleneck" layer zitten die kleiner zijn dan het aantal neuronen in de inputlayer. Op deze manier kan, na het trainen van het netwerk, het aantal dimensies van de data worden verkleind.<br><br>

Een autoencoder kan verdeeld worden in 3 onderdelen:
- Encoder
- Bottleneck layer
- Decoder

De Encoder zorgt voor het verkleinen van het aantal ingevoerde dimensies, de bottleneck layer bevat de hoeveelheid dimensies waar de gebruiker naartoe wil werken, de Decoder vergroot het aantal dimensies weer naar de hoeveelheid van de inputlayer, zodat de loss van input/output berekend kan worden. De encoder en decoder hoeven niet te bestaan uit 1 layer, deze kunnen opgebouwd worden uit verschillende layers. Hier wordt vaak gebruik gemaakt van het 'sandwich' model.<br><br>

Om duidelijk te zien hoe een autoencoder werkt, zullen we deze techniek toepassen bij het trainen van een $k$-nearest neighbors model op de MNIST dataset.

# kNN op de MNIST Dataset

Om een autoencoder te gebruiken moeten we eerst een baseline opstellen, dit doen we door een $k$-nearest neighbors model op te zetten met scikit-learn, en deze te trainen met de MNIST dataset.

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# TODO: print de shapes van x_train, x_test
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


De geïmporteerde MNIST data bevat een array met 2-dimensionale arrays van 28x28 elementen. Elk van deze punten bestaat uit een RGB waarde (0-255). Om makkelijk met de data te kunnen werken moeten we de RGB waarden normaliseren naar waarden tussen 0 en 1. 

In [9]:
#TODO: normaliseer data naar waarden tussen 0 en 1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


Voor het trainen van het model is het handig om de data om te zetten naar 1x784 arrays, in plaats van 2-dimensionale 28x28 arrays.

In [12]:
#TODO: Schrijf een functie om de data deze shape te krijgen:
# x_train: (60000, 784)
# x_test:  (10000, 784)
def twoD_to_oneD(dataset):
    size_dataset = len(dataset)
    flattend_set = np.empty(size_dataset)
    for i in range(size_dataset):
        flattend_set[i] = dataset[i].flatten()

    return flattend_set

#TODO: print de nieuwe shapes van x_train, x_test
x_train_flat = x_train.reshape((60000, 784))
x_test_flat = x_test.reshape((10000, 784))
print(x_test_flat.shape)

(10000, 784)


Met de genormaliseerde data kunnen we een baseline opstellen met $k$-nearest neighbors. Om dit model op te stellen en te trainen kunnen we gebruikmaken van <i>scikit-learn</i>. Gebruik voor het berekenen van de score de <i>.score()</i> functie van het model. TIP: maak bij het berekenen van de score gebruik van 1000 elementen uit <i>x_test</i>. Het berekenen van de score kost erg veel tijd wanneer dit over alle testdata berekend moet worden.

In [ ]:
#TODO: Maak een kNN algoritme
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(x_train)

# Bouwen van een Autoencoder
Voor het trainen van de $k$-nearest neighbors gebruiken we op dit moment alle 784 dimensies/pixels van alle afbeeldingen. Je zult al gemerkt hebben dat het berekenen van de score wat tijd in beslag neemt. Om dit omlaag te brengen kunnen we ervoor kiezen om minder dimensies/pixels te gebruiken. Om dit te bepalen kunnen we een autoencoder gebruiken<br><br>

Omdat een autoencoder een vorm van een neuraal netwerk is, kunnen we <i>Keras</i> gebruiken om een model op te zetten en te trainen.<br><br>

Onderstaande functie <i>display_digits()</i> is voor het uitprinten van de afbeeldingen zodat we het resultaat kunnen vergelijken van de originele afbeeldingen en de afbeeldingen wanneer ze door de autoencoder zijn gehaald.

In [ ]:
def display_digits(x_test, x_decoded, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(x_decoded[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

Voor het bouwen van een neuraal netwerk kan de <i>Sequential([])</i> module van Keras worden gebruikt. Hierin dienen een aantal Dense layers te worden aangemaakt, zodat het netwerk getraind kan worden. We zullen de meest eenvoudige autoencoder opzetten, met een input layer, bottleneck layer, en een output layer. Voor de activatiefuncties kan 'relu' worden gebruikt in de bottleneck layer, en 'sigmoid' in de output layer.<br><br>

In [ ]:
encoding_dim = 32 #op deze manier reduceren we de dimensies bij een factor van 24.5 (784/24.5=32)

#TODO: aanmaken van autoencoder model door middel van Sequential en Dense layers
autoencoder = Sequential()
autoencoder.add(Dense(encoding_dim, activation='relu'))
autoencoder.add(Dense(28, activation='sigmoid'))




Om de autoencoder te trainen dient het model gecompileerd te worden met <i>compile()</i>. Voor de optimizer kan voor 'adadelta' gekozen worden. Omdat we gebruikmaken van data tussen 0 en 1 gebruiken we 'binary_crossentropy' als loss-functie. Wanneer je data hebt dat niet tussen 0 en 1 ligt, kun je gebruikmaken van Mean Squared Error.

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

Om te testen of de autoencoder dichtbij de de originele input zit, kan gebruik worden gemaakt van de <i>.predict()</i> functie van de autoencoder. Als deze functie gebruikt wordt om x_test te voorspellen, kan deze data samen met x_test zelf in de hierboven gedefinieerde <i>display_digits()</i> worden gebruikt om een aantal (10) originele x_test en voorspelde x_test afbeeldingen te printen.

In [ ]:
#TODO: genereer resultaten met .predict() en print afbeeldingen
prediction = model.predict(x_test)
display_digits(prediction, x_test, n=10)

Als de voorspelde afbeeldingen van de autoencoder lijken op de originele afbeeldingen, kunnen we het decoder gedeelte van de autoencoder weglaten, deze is alleen bedoeld voor het trainen. Voor het daadwerkelijk reduceren van de data hebben we alleen de encoder nodig. Omdat onze autoencoder maar 1 layer heeft als encoder, kunnen we layer[0] gebruiken uit ons autoencoder object

In [ ]:
#TODO: maak een encoder Sequential([]) model door de eerste layer te nemen uit het autoencoder model
model = Sequential()
model.add(x.layer[0])

# kNN Met gereduceerde dimensies
Omdat we nu een getrainde encoder hebben, kunnen we deze gebruiken om de dimensies van x_train, x_test te reduceren, zodat we de gereduceerde data te gebruiken om een $k$-nearest neighbors te trainen. Gebruik hier weer hetzelfde aantal neighbors en bereken de score van het model ook weer met 1000 waarden uit de testdata.

In [ ]:
#TODO: predict x_train_encoded en x_test_encoded met behulp van de encoder
x_train_encoded = model.predict(x_train)
x_test_encoded  = model.predict(x_test)

In [ ]:
#TODO: train een kNN met x_train_encoded en x_test_encoded met hetzelfde aantal neighbors als in de baseline

# Opdracht: Deep Autoencoder
Probeer met deze tools een nieuwe autoencoder te maken, door gebruik te maken van meerdere layers in het encoder en decoder gedeelte. Dit wordt een Deep Autoencoder genoemd. Test de werking van je autoencoder door met behulp van een encoder de data weer in een $k$-nearest neighbors te stoppen. Denk bij het maken van meerdere layers aan het 'sandwich' model.

In [ ]:
#TODO: Deep Autoencoder

In [ ]:
#TODO: print de resultaat van de autoencoder

In [ ]:
#TODO: Maak een encoder

In [ ]:
#TODO: Nearest Neighbors

### Bronnen
https://blog.keras.io/building-autoencoders-in-keras.html<br>
https://keras.io/models/sequential/<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html<br>
https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798<br>
https://www.youtube.com/watch?v=H1AllrJ-_30<br>
https://www.researchgate.net/post/Should_the_output_function_for_outer_layer_and_activation_function_of_hidden_layer_in_auto_encoder_be_same<br>